# German Credit data

- 신용평가 모델
- 대출 신청자 1000명의 샘플데이터 => 실제 대출 승인여부 (good, bad)


- https://www.kaggle.com/datasets/uciml/german-credit

- https://www.kaggle.com/datasets/uciml/german-credit?resource=download

- Age: 나이 
- Sex: 성별
- Job: 직업
- Housing: 거주 형태(자가, 임대, 무상거주)
- Saving accounts: 저축 계좌 잔고
- Checking account: 입출금 계좌 잔고
- Credit amount: 대출 금액
- Duration: 대출 기간
- Purpose: 대출 목적
- Risk: 대출 위험도(상환 불이행 가능성)
- 
- Age (numeric)
- Sex (text: male, female)
- Job (numeric: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)
- Housing (text: own, rent, or free)
- Saving accounts (text - little, moderate, quite rich, rich)
- Checking account (numeric, in DM - Deutsch Mark)
- Credit amount (numeric, in DM)
- Duration (numeric, in month)
- Purpose(text: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others
- Risk (Value target - Good or Bad Risk)

# German credit data

신용평가모델

-대출 신청자 1000명의 샘플 데이터 => 실제 대출 승인여부 (good, bad)

It is worse to class a customer as good when they are bad,
than it is to class a customer as bad when they are good.

분석의 목적:
- 고객 데이터를 기반으로 => 이 사람의 대출 승인 여부를 판단하는 로직(기준)을 좀 알려줘.

종속변수(우리가 알고 싶은 결과):
- Risk(good, bad)

가설: 
=> 각각의 컬럼들이 -> Risk에 미치는 영향을 파악하면 -> 피처 엔지니어링에 활용 -> 모델링에 활용

EDA : 데이터를 보면서 종속 변수에 영향을 미치는 중요한 컬럼들을 발견하기 위해

(1) 저축계좌(적금계좌) / 예금계좌가 없거나 적은 사람은 신용에 문제가 있을 가능성이 높다.
(2) 대출목적 (TV) 신용에 문제가 있을 것이다.

In [ ]:
import pandas as pd #To work with dataset
import numpy as np #Math library
import seaborn as sns #Graph library that use matplot in background
import matplotlib.pyplot as plt #to plot some parameters in seaborn